In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

ModuleNotFoundError: No module named 'setter'

In [ ]:

hydrogen_res_name = 'H2'
FF_file_hydrogen = 'h2_ff.xml'
hydrogen_FF = Forcefield(forcefield_files=FF_file_hydrogen)
hydrogen_mb = mb.load('[HH]', smiles=True)

hydrogen_mb.name = hydrogen_res_name

hydrogen_typed=hydrogen_FF.apply(hydrogen_mb)



constrain_mol = ConstrainedMolecule(hydrogen_typed)
constrain_mol.solve()
hydrogen_optimized_mb=mb.clone(hydrogen_mb)
hydrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(hydrogen_optimized_mb.xyz)

In [ ]:
hydrogen_optimized_mb.visualize()

In [ ]:

nitrogen_res_name = 'N2'
FF_file_nitrogen = 'dinitrogen_ff.xml'
nitrogen_FF = Forcefield(forcefield_files=FF_file_nitrogen)
nitrogen_mb = mb.load('N#N', smiles=True)

nitrogen_mb.name = nitrogen_res_name

nitrogen_typed=nitrogen_FF.apply(nitrogen_mb)



constrain_mol = ConstrainedMolecule(nitrogen_typed)
constrain_mol.solve()
nitrogen_optimized_mb=mb.clone(nitrogen_mb)
nitrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(nitrogen_optimized_mb.xyz)

In [ ]:
nitrogen_optimized_mb.visualize()

## Creating .psf (Topology) and .inp (CHARMM potential file) using mosdef_charmm_writer


In [3]:
#random box to be used in mosdef_charmm_writer
hydrogen_box_bias  = mb.fill_box(compound=[hydrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

nitrogen_box_bias  = mb.fill_box(compound=[nitrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'hydrogen_optimized_mb' is not defined

In [ ]:
mixed_box  = mb.fill_box(compound=[hydrogen_optimized_mb, nitrogen_box_bias ],
                             n_compounds=[1, 1] ,
                            box=[1.0, 1.0, 1.0] )

In [ ]:
FF_Dict = {hydrogen_optimized_mb.name:FF_file_hydrogen, nitrogen_optimized_mb.name:FF_file_nitrogen }

residues_List = [hydrogen_optimized_mb.name, nitrogen_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(mixed_box ,
                            'overall_bias',
                            FF_filename ="overall_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

### Now we have the .psf (topology) file and .inp (charmm potential) file

## Creating CP2K MC simulation

In [ ]:
molecule_list=[hydrogen_optimized_mb, nitrogen_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed


In [ ]:
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[2,2],[2,2]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'N':'DZVP-MOLOPT-GTH','H':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NPT',seed=1,project_name='NH3_NPT_GEMC',restart='FALSE',pressure=1*u.bar)

In [ ]:
q.mc_initialization()

In [ ]:

q.topology_filename=['hydrogen_bias.psf', 'nitrogen_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1],[1],[1],[1],[1]],[[1],[1],[1],[1],[1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1],[1],[1],[1]],[[1],[1],[1],[1]]]

q.charmm_potential_file='overall_charmm_bias.inp'

In [ ]:
setter.mc_files(q)

In [ ]:
!cp2k.popt -i GEMC_NPT_box1.inp -i GEMC_NPT_box2.inp